# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [5]:
from pyspark.sql.functions import col, count, countDistinct, lit, when

Create our session as described in the tutorials

In [6]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [7]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [8]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [9]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [11]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

#### Subset patent table on columns PATENT AND POSTATE, new table "**patents_subset**"

In [12]:
patents_subset = patents.select(['PATENT','POSTATE'])

#### Join table "**Citation**" with  "**patents_subset**" on CITED column resulting in **TABLE1**

In [13]:
table1 = citations.join(patents_subset,citations.CITED == patents_subset.PATENT)\
                  .withColumnRenamed('POSTATE','CITED_STATE')
table1 = table1.drop('PATENT')

#### Take distinct CITING values from "**citations**" table 
#### and join the unique CITING values with "**patents_subset**" resulting in **TABLE2**

In [14]:
unique_citing = citations.select('CITING').distinct()
table2 = unique_citing.join(patents_subset,unique_citing.CITING == patents_subset.PATENT)\
                      .withColumnRenamed('POSTATE','CITING_STATE') 
table2 = table2.drop('PATENT')

#### Join **TABLE1** and **TABLE2** to get an intermediate table

In [15]:
table3 = table1.join(table2,['CITING'])

#### Intermediate table consists of Citing, Citing_state, Cited and Cited_state

In [17]:
table3.show(5)

+-------+-------+-----------+------------+
| CITING|  CITED|CITED_STATE|CITING_STATE|
+-------+-------+-----------+------------+
|3858527|3638586|         CA|        null|
|3858527|3699902|         OH|        null|
|3858527|3602157|         TX|        null|
|3858597|3815160|         NY|          MT|
|3858597|3675252|         AZ|          MT|
+-------+-------+-----------+------------+
only showing top 5 rows



#### Creating new boolean column "**SAME_STATE**" to check if the citing is within same state using groupby <br\> and counting using aggregation operation

In [18]:
table4 = table3.withColumn("SAME_STATE",table3.CITING_STATE == table3.CITED_STATE)\
               .groupby("CITING","SAME_STATE")\
               .agg(count("*"))

In [19]:
table4 = table4.withColumnRenamed("count(1)","SAME_STATE_COUNT")

#### Filtering column "**SAME_STATE**" when it is True

In [20]:
table4 = table4.filter(table4.SAME_STATE == True)

#### Join the final count of same state citations with the orginal patent table 

In [21]:
patents_new = patents.join(table4,table4.CITING == patents.PATENT).select(patents["*"],table4["SAME_STATE_COUNT"])

#### Sorting the table based on column "**SAME_STATE_COUNT**" and printing the top 10

In [22]:
patents_new_sorted = patents_new.sort(['SAME_STATE_COUNT'],ascending=False)

In [23]:
patents_new_sorted.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE_COUNT|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|             125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|             103|
